In [12]:
from pettingzoo.sisl import multiwalker_v9
env = multiwalker_v9.parallel_env()
obs = env.reset()

In [13]:
print(env.agents)

['walker_0', 'walker_1', 'walker_2']


In [14]:
print(env.observation_spaces)

{'walker_0': Box(-inf, inf, (31,), float32), 'walker_1': Box(-inf, inf, (31,), float32), 'walker_2': Box(-inf, inf, (31,), float32)}


In [15]:
print(env.action_spaces)

{'walker_0': Box(-1.0, 1.0, (4,), float32), 'walker_1': Box(-1.0, 1.0, (4,), float32), 'walker_2': Box(-1.0, 1.0, (4,), float32)}


In [16]:
print(env.action_spaces['walker_0'])

Box(-1.0, 1.0, (4,), float32)


In [17]:
print(env.num_agents)

3


In [18]:
print(obs)

{'walker_0': array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  1.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  1.6236708e-01,  1.6546237e-01,
        4.4486943e-01,  4.7198775e-01,  5.1494205e-01,  5.8084798e-01,
        6.8370843e-01,  8.5414934e-01,  9.1396463e-01,  8.7226135e-01,
        0.0000000e+00,  0.0000000e+00,  3.4050453e-01, -2.2062914e-04,
        3.3950117e-01,  8.1902765e-02, -1.3480984e-03], dtype=float32), 'walker_1': array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  1.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  1.6236708e-01,  1.6546237e-01,
        4.4349247e-01,  4.6802202e-01,  5.1206982e-01,  5.8297271e-01,
        6.8496138e-01,  8.3889270e-01,  9.1396481e-

In [22]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.distributions import Categorical                          #创建以参数probs为标准的类别分布，样本是来自 “0 … K-1” 的整数，其中 K 是probs参数的长度。
                                                                     #也就是说，按照传入的probs中给定的概率，在相应的位置处进行取样，取样返回的是该位置的整数索引
from torch.distributions import MultivariateNormal                   #multivariate-normal 多元正态分布
import numpy as np

class MemoryBuffer:
    '''Simple buffer to collect experiences and clear after each update.'''
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.dones = []
        self.state_values = []
    
    def clear_buffer(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.dones[:]
        del self.state_values[:]
    
    def get_ordered_trajectories(self, n_agents=env.num_agents):#trajectories - 轨迹
        ordered_actions = torch.FloatTensor()
        ordered_states = torch.FloatTensor()
        ordered_logprobs = torch.FloatTensor()
        ordered_rewards = []
        ordered_dones = []
        
        #把一个个action、state、prob入栈（但为什么没有reward？）
        actions = torch.stack(self.actions)                
        states = torch.stack(self.states)
        logprobs = torch.stack(self.logprobs)

        self.ordered_actions = torch.FloatTensor()
        
        for index in range(actions.shape[1]):
            if n_agents !=None and n_agents == index+1:
                break
                
            #在pytorch中，常见的拼接函数主要是两个，分别是：
            # stack()
            # cat()
            
            ordered_states = torch.cat((ordered_states, states[:, index]), 0)
            ordered_actions = torch.cat((ordered_actions, actions[:, index]), 0)
            ordered_logprobs = torch.cat((ordered_logprobs, logprobs[:, index]), 0)
            #print("in Memory.get_order_trajectories(),\t self.rewards = ",self.rewards,'\t')
            #print("in Memory.get_order_trajectories(),\t self.dones = ",self.dones,'\t')
            ordered_rewards.extend(np.asarray(self.rewards)[:, index])
            ordered_dones.extend(np.asarray(self.dones)[:, index])

        return ordered_states, ordered_actions, ordered_logprobs, ordered_rewards, ordered_dones
        

        
class ActorCritic(nn.Module):
    def __init__(self, state_size, action_size, action_std=0.5, hidden_size=32, low_policy_weights_init=True):
        super().__init__()
        
        self.actor_fc1 = nn.Linear(state_size, 4*hidden_size)
        self.actor_fc2 = nn.Linear(4*hidden_size, 4*hidden_size)
        self.actor_fc3 = nn.Linear(4*hidden_size, 2*hidden_size)
        self.actor_mu = nn.Linear(2*hidden_size, action_size)
        self.actor_sigma = nn.Linear(2*hidden_size, action_size)
        #actor的神经网络结构：
        #输入 state --> 2*hidden_size个神经元-->2*hidden_size个神经元-->hidden_size个神经元-->action_size个神经元-->sigma函数
        
        self.critic_fc1 = nn.Linear(state_size, 4*hidden_size)
        self.critic_fc2 = nn.Linear(4*hidden_size, 4*hidden_size)
        self.critic_fc3 = nn.Linear(4*hidden_size, 2*hidden_size)
        self.critic_value = nn.Linear(2*hidden_size, 1)
        #critic的神经网络结构
        #输入 state --> 2*hidden_size个神经元-->2*hidden_size个神经元-->hidden_size个神经元-->1个神经元
        
        self.distribution = torch.distributions.Normal
        self.action_var = torch.full((action_size,), action_std*action_std)
        
        # Boosts training performance in the beginning
        if low_policy_weights_init:
            with torch.no_grad():
                self.actor_mu.weight.mul_(0.01)

                
    def forward(self, state):
        #用tanh作为激活函数
        x = torch.tanh(self.actor_fc1(state))
        x = torch.tanh(self.actor_fc2(x))
        x = torch.tanh(self.actor_fc3(x))
        mu = torch.tanh(self.actor_mu(x))                  #对应正态分布的均值miu
        
        #用softplus作为激活函数
        sigma = F.softplus(self.actor_sigma(x))/3            #对应正态分布的方差sigma
        v = torch.tanh(self.critic_fc1(state))
        v = torch.tanh(self.critic_fc2(v))
        v = torch.tanh(self.critic_fc3(v))
        state_value = self.critic_value(v)
        #print("in function:forward,\tmu,sigma,state_value ==",mu, sigma, state_value)
        return mu, sigma, state_value 

    def act(self, state):
        '''Choose action according to the policy.'''
        #state -> nn == action_mu
        action_mu, action_sigma, state_value = self.forward(state)
        # action_mu 
        action_var = self.action_var.expand_as(action_mu)
        cov_mat = torch.diag_embed(action_var)
        dist = MultivariateNormal(action_mu, cov_mat)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        #print("in function:act,\taction.detach(), log_prob.detach()==",action.detach(), log_prob.detach())
        return action.detach(), log_prob.detach()
    
    def evaluateStd(self, state, action):
        '''Evaluate action using learned std value for distribution.'''
        action_mu, action_sigma, state_value = self.forward(state)
        m = self.distribution(action_mu.squeeze(), action_sigma.squeeze())
        log_prob = m.log_prob(action)

        return log_prob, state_value

    def evaluate(self, state, action):
        '''Evaluate action for a given state.'''   
        action_mean, action_var, state_value = self.forward(state)
        #print("in function evaluate:action_mean,action_var=",action_mean,action_var)
        action_var = self.action_var.expand_as(action_mean)
        cov_mat = torch.diag_embed(action_var)
        
        dist = MultivariateNormal(action_mean, cov_mat)
        
        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        
        return action_logprobs, torch.squeeze(state_value), dist_entropy

class PPO():
    '''Proximal Policy Optimization algorithm.'''
    def __init__(self, state_size, action_size, lr=1e-4, gamma=0.99, epsilon_clip=0.2, epochs=20, action_std=0.5):

        self.state_size = state_size
        self.action_size = action_size
        self.lr = lr
        self.gamma  = gamma
        self.epsilon_clip = epsilon_clip
        self.K_epochs = epochs

        self.policy = ActorCritic(self.state_size, self.action_size, action_std, hidden_size=128)
        self.policy_old = ActorCritic(self.state_size, self.action_size, action_std, hidden_size=128)

        self.MseLoss = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=self.lr, betas=(0.9, 0.999))

        self.episode = 0
    
    def select_action(self, state):
        '''Get action using state in numpy format'''
        # state = torch.FloatTensor(state.reshape(1, -1))
        state = torch.FloatTensor(state)
        #print("in function:select_action,\tself.policy_old.act(state)==",self.policy_old.act(state))
        return self.policy_old.act(state)

    def update(self, memory):
        '''Update agent's network using collected set of experiences.'''
        states, actions, log_probs, rewards, dones = memory.get_ordered_trajectories(env.num_agents)

        discounted_rewards = []
        discounted_reward = 0
        for i in reversed(range(len(rewards))):
            if dones[i] == True:
                discounted_reward = 0  
            discounted_reward = rewards[i] + self.gamma*discounted_reward
            discounted_rewards.insert(0, discounted_reward)
        
        discounted_rewards = torch.tensor(discounted_rewards)
        # old_state_values = torch.stack(state_values, 1).detach()
        # advantages = discounted_rewards - old_state_values.detach().squeeze()
        # advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-5)
        discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-5)
        
        # states = torch.squeeze(torch.stack(states), 1).detach()
        # actions = torch.squeeze(torch.stack(actions), 1).detach()
        # old_log_probs = torch.squeeze(torch.stack(log_probs), 1).detach()

        states = states.detach()
        actions = actions.detach()
        old_log_probs = log_probs.detach()


        for epoch in range(self.K_epochs):

            new_log_probs, state_values, dist_entropy = self.policy.evaluate(states, actions)

            new_log_probs = new_log_probs.squeeze()
            advantages = discounted_rewards - state_values.detach().squeeze()
            ratios = torch.exp(new_log_probs - old_log_probs.detach())
            ratios_clipped = torch.clamp(ratios, min=1-self.epsilon_clip, max=1+self.epsilon_clip)
            loss = -torch.min(ratios*advantages, ratios_clipped*advantages)+ 0.5*self.MseLoss(state_values, discounted_rewards) - 0.01*dist_entropy
            loss = torch.tensor(loss,dtype=float,requires_grad=True)
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()

        self.policy_old.load_state_dict(self.policy.state_dict())

In [23]:
import torch                           
import numpy as np          
from collections import deque                                   #队列的数据结构
import time                     
from torch.utils.tensorboard import SummaryWriter               #这个包的作用是让使用者可以通过简单的界面用PyTorch获取events，
from pettingzoo.sisl import multiwalker_v9                      #并且在tensorboard中显示。


n_agents = 3
n_episodes = 12000
max_steps = 300
update_interval = 1200/n_agents
log_interval = 10
solving_threshold = 30
time_step = 0
render = True
train = True
pretrained = False
tensorboard_logging = True

env_name = "multiwalker_v9_new3"
env = multiwalker_v9.parallel_env(forward_reward=50,position_noise=0, angle_noise=0,fall_reward=-10)

obsagents= env.reset()
action_size = 4
states = obs
state_size = 31

print("State size: ", state_size)
print("Action size: ", action_size)

scores = deque(maxlen=log_interval)
max_score = -1000
episode_lengths = deque(maxlen=log_interval)
rewards =  []

memory = MemoryBuffer()

agent = PPO(state_size, action_size)

if not train:
    agent.policy_old.eval()
else:
    print('logs/'+env_name+'_'+str(time.time()))

if pretrained:
    agent.policy_old.load_state_dict(torch.load('./'+env_name+'_model_best_old.pth'))
    agent.policy.load_state_dict(torch.load('./'+env_name+'_model_best_old.pth'))


State size:  31
Action size:  4
logs/multiwalker_v9_new3_1661347360.0029879


F:\file\pettingzoo_rl_practice\lib\site-packages\gym\utils\seeding.py:63: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high, size, dtype])` instead.
  deprecation(


In [24]:
doc = open('record3.txt',mode='r+',buffering=1)
for n_episode in range(1, n_episodes+1):
    obs = env.reset()
    states = list(obs.values())
    #state = states[0]
    
    states = torch.FloatTensor(states)
    # print("States shape: ", states.shape)
    # state = torch.FloatTensor(state.reshape(1, -1))
    
    episode_length = 0
    episodic_rewards = []
    done = 0
    total_rewards = []
    for t in range(max_steps):
        
        time_step += 1

        actions, log_probs = agent.select_action(states)
        #print("actions ==",actions,file=doc)

        states = torch.FloatTensor(states)
        memory.states.append(states)
        memory.actions.append(actions)
        memory.logprobs.append(log_probs)

        # actions = []
        # ## Unity env style
        # for agent_id in range(0,20):
        #     actions.append(action.data.numpy().flatten())

        obs, rewards, dones, infos = env.step(dict(zip(env.agents,actions.data.numpy().clip(-1,1))))          # send all actions to tne environment
        states = list(dict(obs).values())
        rewards = list(dict(rewards).values())
        dones = list(dict(dones).values()) 
        #print("obs, rewards, dones, infos = \n",obs,'\n', rewards,'\n', dones,'\n', infos,file=doc)
        

        
        
        #state = states[0]
        #reward = rewards[0]
        done = dones[0]

        # state, reward, done, _ = env.step(action.data.numpy().flatten())


        memory.rewards.append(rewards)
        #print('in class:memory,memory.rewards=',memory.rewards)
        memory.dones.append(dones)
        episodic_rewards.append(rewards)
        state_value = 0
        
        if render:
            env.render()
             #image = env.render(mode = 'rgb_array')
             #if time_step % 2 == 0:
             #writerImage.append_data(image)

        if train:
            if time_step % update_interval == 0:
                
                agent.update(memory)
                time_step = 0
                memory.clear_buffer()

        episode_length = t

        if done:
            break
            
    
    episode_lengths.append(episode_length)
    total_reward = np.sum(episodic_rewards)/n_agents
    print('total_rewards =',total_reward,file=doc)
    scores.append(total_reward)
    
    if train:
        if n_episode % log_interval == 0:
            print("Episode: ", n_episode, "\t Avg. episode length: ", np.mean(episode_lengths), "\t Avg. score: ", np.mean(scores))

            if np.mean(scores) > solving_threshold:
                print("Environment solved, saving model")
                torch.save(agent.policy_old.state_dict(), 'PPO_model_solved_{}.pth'.format(env_name))
        
        if n_episode % 100 == 0:
            print("Saving model after ", n_episode, " episodes")
            torch.save(agent.policy_old.state_dict(), '{}_model_{}_episodes.pth'.format(env_name, n_episode))
            
        if total_reward > max_score:
            print("Episode: ", n_episode,'\t')
            print("Saving improved model")
            max_score = total_reward
            torch.save(agent.policy_old.state_dict(), '{}_model_best.pth'.format(env_name))

        if tensorboard_logging:
            print('Score', {'Score':total_reward, 'Avg._Score': np.mean(scores)}, n_episode)
            print('Episode_length', {'Episode_length':episode_length, 'Avg._Episode length': np.mean(episode_lengths)}, n_episode)
            #writer.add_scalars('Score', {'Score':total_reward, 'Avg._Score': np.mean(scores)}, n_episode)
            #writer.add_scalars('Episode_length', {'Episode_length':episode_length, 'Avg._Episode length': np.mean(episode_lengths)}, n_episode)
    
    else:
        print("Episode: ", n_episode, "\t Episode length: ", episode_length, "\t Score: ", total_reward)
        
    total_reward = 0
print(scores)

Episode:  1 	
Saving improved model
Score {'Score': -110.3982995450498, 'Avg._Score': -110.3982995450498} 1
Episode_length {'Episode_length': 116, 'Avg._Episode length': 116.0} 1
Episode:  2 	
Saving improved model
Score {'Score': -6.314613272746338, 'Avg._Score': -58.35645640889807} 2
Episode_length {'Episode_length': 99, 'Avg._Episode length': 107.5} 2
Score {'Score': -18.674550180633933, 'Avg._Score': -45.129154332810025} 3
Episode_length {'Episode_length': 66, 'Avg._Episode length': 93.66666666666667} 3
Score {'Score': -231.45725294947644, 'Avg._Score': -91.71117898697662} 4
Episode_length {'Episode_length': 75, 'Avg._Episode length': 89.0} 4


C:\Users\25243\AppData\Local\Temp\ipykernel_13468\4011111466.py:202: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = torch.tensor(loss,dtype=float,requires_grad=True)


Score {'Score': -12.2851859033107, 'Avg._Score': -75.82598037024344} 5
Episode_length {'Episode_length': 63, 'Avg._Episode length': 83.8} 5
Episode:  6 	
Saving improved model
Score {'Score': 78.27139973640415, 'Avg._Score': -50.143083685802175} 6
Episode_length {'Episode_length': 68, 'Avg._Episode length': 81.16666666666667} 6
Score {'Score': -145.4194817940395, 'Avg._Score': -63.75399770126465} 7
Episode_length {'Episode_length': 69, 'Avg._Episode length': 79.42857142857143} 7
Score {'Score': -120.69490611553208, 'Avg._Score': -70.87161125304807} 8
Episode_length {'Episode_length': 66, 'Avg._Episode length': 77.75} 8
Score {'Score': -105.77677617470431, 'Avg._Score': -74.74996291100987} 9
Episode_length {'Episode_length': 62, 'Avg._Episode length': 76.0} 9
Episode:  10 	 Avg. episode length:  74.5 	 Avg. score:  -66.29674782107281
Score {'Score': 9.782187988360809, 'Avg._Score': -66.29674782107281} 10
Episode_length {'Episode_length': 61, 'Avg._Episode length': 74.5} 10
Score {'Score

Score {'Score': -23.266450191537626, 'Avg._Score': -50.67006505746399} 59
Episode_length {'Episode_length': 72, 'Avg._Episode length': 74.2} 59
Episode:  60 	 Avg. episode length:  76.3 	 Avg. score:  -47.687616404611646
Score {'Score': -115.84361088617395, 'Avg._Score': -47.687616404611646} 60
Episode_length {'Episode_length': 65, 'Avg._Episode length': 76.3} 60
Score {'Score': -94.90447218219447, 'Avg._Score': -65.30804707777381} 61
Episode_length {'Episode_length': 61, 'Avg._Episode length': 72.6} 61
Score {'Score': -661.9569453721246, 'Avg._Score': -133.1710340261149} 62
Episode_length {'Episode_length': 299, 'Avg._Episode length': 96.5} 62
Score {'Score': -73.14891656239836, 'Avg._Score': -132.98121252330026} 63
Episode_length {'Episode_length': 70, 'Avg._Episode length': 98.6} 63
Score {'Score': -163.75483199954041, 'Avg._Score': -140.9082945552655} 64
Episode_length {'Episode_length': 74, 'Avg._Episode length': 97.4} 64
Score {'Score': -127.04957420627291, 'Avg._Score': -148.278

Episode:  112 	
Saving improved model
Score {'Score': 301.6960963606836, 'Avg._Score': -56.873663336659526} 112
Episode_length {'Episode_length': 73, 'Avg._Episode length': 72.3} 112
Score {'Score': -34.57354910671733, 'Avg._Score': -54.64168621165063} 113
Episode_length {'Episode_length': 57, 'Avg._Episode length': 70.9} 113
Score {'Score': 133.09746255477248, 'Avg._Score': -39.09478948824113} 114
Episode_length {'Episode_length': 68, 'Avg._Episode length': 72.0} 114
Score {'Score': -6.964202870925192, 'Avg._Score': -14.556394858285827} 115
Episode_length {'Episode_length': 95, 'Avg._Episode length': 75.1} 115
Score {'Score': 35.107665732503, 'Avg._Score': -0.3419858720153961} 116
Episode_length {'Episode_length': 64, 'Avg._Episode length': 75.4} 116
Score {'Score': -126.50426771491782, 'Avg._Score': 1.6971500845004712} 117
Episode_length {'Episode_length': 104, 'Avg._Episode length': 75.8} 117
Score {'Score': 5.581263837714766, 'Avg._Score': -10.622600711882187} 118
Episode_length {'

...

Episode:  11760 	 Avg. episode length:  71.2 	 Avg. score:  -23.966795378675144
Score {'Score': 60.18755656977472, 'Avg._Score': -23.966795378675144} 11760
Episode_length {'Episode_length': 65, 'Avg._Episode length': 71.2} 11760
Score {'Score': -124.49696879833944, 'Avg._Score': -29.23772794504969} 11761
Episode_length {'Episode_length': 59, 'Avg._Episode length': 72.0} 11761
Score {'Score': -45.630550459027226, 'Avg._Score': -22.39094666888325} 11762
Episode_length {'Episode_length': 127, 'Avg._Episode length': 80.3} 11762
Score {'Score': -90.4825675735871, 'Avg._Score': -34.69378207127261} 11763
Episode_length {'Episode_length': 63, 'Avg._Episode length': 80.5} 11763
Score {'Score': -178.23571121630584, 'Avg._Score': -46.63261364990232} 11764
Episode_length {'Episode_length': 62, 'Avg._Episode length': 79.1} 11764
Score {'Score': -91.11649771531425, 'Avg._Score': -35.25476731949802} 11765
Episode_length {'Episode_length': 55, 'Avg._Episode length': 76.5} 11765
Score {'Score': -9.1997

Score {'Score': -93.5653656527091, 'Avg._Score': -78.15884675139881} 11812
Episode_length {'Episode_length': 40, 'Avg._Episode length': 66.1} 11812
Score {'Score': 16.604367159306786, 'Avg._Score': -66.68680705556969} 11813
Episode_length {'Episode_length': 72, 'Avg._Episode length': 69.2} 11813
Score {'Score': -181.83544181287297, 'Avg._Score': -93.65000284020802} 11814
Episode_length {'Episode_length': 73, 'Avg._Episode length': 66.8} 11814
Score {'Score': -89.370430999746, 'Avg._Score': -82.6507812762479} 11815
Episode_length {'Episode_length': 46, 'Avg._Episode length': 66.5} 11815
Score {'Score': -63.13432266314848, 'Avg._Score': -71.06783684583706} 11816
Episode_length {'Episode_length': 54, 'Avg._Episode length': 64.7} 11816
Score {'Score': -263.70160539944965, 'Avg._Score': -92.65345324816505} 11817
Episode_length {'Episode_length': 82, 'Avg._Episode length': 67.1} 11817
Score {'Score': -217.34604381024835, 'Avg._Score': -135.95575941346286} 11818
Episode_length {'Episode_lengt

Score {'Score': -46.152368932962645, 'Avg._Score': -48.38086230556184} 11865
Episode_length {'Episode_length': 64, 'Avg._Episode length': 75.4} 11865
Score {'Score': -152.95109982291874, 'Avg._Score': -59.15417908007913} 11866
Episode_length {'Episode_length': 67, 'Avg._Episode length': 73.7} 11866
Score {'Score': 140.17042157550645, 'Avg._Score': -37.499458753814416} 11867
Episode_length {'Episode_length': 66, 'Avg._Episode length': 72.4} 11867
Score {'Score': -323.8030946254731, 'Avg._Score': -69.15775445476183} 11868
Episode_length {'Episode_length': 73, 'Avg._Episode length': 71.6} 11868
Score {'Score': 104.54798338313907, 'Avg._Score': -55.326345520714995} 11869
Episode_length {'Episode_length': 71, 'Avg._Episode length': 72.6} 11869
Episode:  11870 	 Avg. episode length:  73.8 	 Avg. score:  -52.96934780726839
Score {'Score': 31.07127264142029, 'Avg._Score': -52.96934780726839} 11870
Episode_length {'Episode_length': 70, 'Avg._Episode length': 73.8} 11870
Score {'Score': -244.790

Score {'Score': -105.07708614071227, 'Avg._Score': -43.152130377789405} 11917
Episode_length {'Episode_length': 299, 'Avg._Episode length': 96.5} 11917
Score {'Score': -59.804211358229544, 'Avg._Score': -39.94109295060245} 11918
Episode_length {'Episode_length': 51, 'Avg._Episode length': 94.7} 11918
Score {'Score': -197.37497960527745, 'Avg._Score': -53.577287713686715} 11919
Episode_length {'Episode_length': 70, 'Avg._Episode length': 94.8} 11919
Episode:  11920 	 Avg. episode length:  92.8 	 Avg. score:  -76.85536669194707
Score {'Score': -246.41043434540438, 'Avg._Score': -76.85536669194707} 11920
Episode_length {'Episode_length': 57, 'Avg._Episode length': 92.8} 11920
Score {'Score': 2.4930134663978243, 'Avg._Score': -77.84538247932997} 11921
Episode_length {'Episode_length': 57, 'Avg._Episode length': 90.4} 11921
Score {'Score': 53.02132550006102, 'Avg._Score': -53.737538888429675} 11922
Episode_length {'Episode_length': 56, 'Avg._Episode length': 90.2} 11922
Score {'Score': -135

Score {'Score': -17.67645047356693, 'Avg._Score': -120.51242359230923} 11969
Episode_length {'Episode_length': 54, 'Avg._Episode length': 69.0} 11969
Episode:  11970 	 Avg. episode length:  70.7 	 Avg. score:  -115.21145025330293
Score {'Score': -88.64428016046683, 'Avg._Score': -115.21145025330293} 11970
Episode_length {'Episode_length': 70, 'Avg._Episode length': 70.7} 11970
Score {'Score': 145.7415069142977, 'Avg._Score': -105.55642470158641} 11971
Episode_length {'Episode_length': 94, 'Avg._Episode length': 67.6} 11971
Score {'Score': -91.73971219609199, 'Avg._Score': -103.72796754725285} 11972
Episode_length {'Episode_length': 63, 'Avg._Episode length': 62.1} 11972
Score {'Score': -116.2200495849054, 'Avg._Score': -102.9540378147116} 11973
Episode_length {'Episode_length': 59, 'Avg._Episode length': 60.2} 11973
Score {'Score': -103.80644915004585, 'Avg._Score': -99.11412283840288} 11974
Episode_length {'Episode_length': 43, 'Avg._Episode length': 59.9} 11974
Score {'Score': 146.93

In [ ]:

print('this is a record!\n',file=doc)